#### Load Train Set

In [1]:
import pandas as pd
import numpy as np
from lib.data import load_dataset

plain = False # Quero só a imagem serializada?
if plain:
    path = 'dts/dts_plain/'
else:
    path = 'dts/dts_all/'

rs = 2
x_train, y_train, x_valid, y_valid = load_dataset(path + 'train.csv', rs=rs)

#### Definição da pipeline
- Definição do normalizador
- Definição do seletor de features
- Definição do random forest classifier

In [4]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

sel = SelectFromModel(estimator=LogisticRegression(random_state=rs, max_iter = 2000).fit(x_train, y_train)).fit(x_train, y_train)
std = StandardScaler()

clf = Pipeline([
    ('standardscaler', std),
    ('feature_selection', sel),
    ('classification', SVC(random_state = rs, tol=1e-2))
])


d:\ProgramData\Anaconda3\envs\ACA\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\ProgramData\Anaconda3\envs\ACA\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

#### Treino do modelo

In [5]:
x = pd.DataFrame(x_train)
y = np.array(y_train)
clf.fit(x, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('feature_selection',
                 SelectFromModel(estimator=LogisticRegression(max_iter=2000,
                                                              random_state=2))),
                ('classification', SVC(random_state=2, tol=0.01))])

#### Validation

In [6]:
y_pred = clf.predict(x_valid)

#### Display das metrics (Accuracy, RoC, precision, recall)

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score



accuracy = accuracy_score(y_true=y_valid, y_pred=y_pred)
roc = roc_auc_score(y_valid, y_pred)
pre = precision_score(y_true=y_valid, y_pred=y_pred)
rec = recall_score(y_true=y_valid, y_pred=y_pred)


print("Metrics: \n Acc: {acc:.3f}, ROC: {roc:.2f}\n PRE: {pre:.2f}, REC: {rec:.2f}".format(acc=accuracy,roc=roc,pre=pre,rec=rec))


Metrics: 
 Acc: 0.979, ROC: 0.98
 PRE: 0.98, REC: 0.98


##### Load dataset de teste e exporta resultado para um arquivo .csv

In [ ]:
df2 = pd.read_csv(path + "/test.csv")

x_test = df2.iloc[:, :-1]
y_pred = clf.predict(x_test)

result = pd.DataFrame(data={
        "Id": range(y_pred.shape[0]),
        "Category": y_pred.astype(int)
    }, index=None)

result.to_csv("result.csv", index=None)